### Find the cheapest path

##### Goal
 - Find the cheapest path from start to finish. 
 
##### Input:  
- MxN matrix of weights
- start point
- finish point  

##### Output:
- list of steps from start to finish. Example: ['down', 'down', 'right', 'right', 'up', 'up']


In [ ]:
# Version 1
from collections import namedtuple
from itertools import chain,filterfalse

Path = namedtuple('Path', ['val', 'dir'])

class Maze(object):
                      
    steps = [ (0,1,'right'), (1,0,'down'), (-1,0,'up'), (0,-1,'left') ]
                      
    def __init__(self, toll_map, start, finish):
        self.toll_map = toll_map
        self.start = start
        self.finish = finish
        self.ys = len(toll_map)
        self.xs = len(toll_map[0])
        bv = Path(float('inf'), '')
        self.path = [ [bv]*self.xs for i in range(0,self.ys) ]
        self.path[start[0]][start[1]] = Path(0, '')
    
    def is_valid(self, y, x):
        return y >= 0 and x >= 0 and y < self.ys and x < self.xs
    
    def update_step(self, y, x, step):
        ny, nx = y + step[0], x + step[1]
        if self.is_valid(ny, nx): 
            next_cost = self.toll_map[y][x] + self.path[y][x].val
            if self.path[ny][nx].val > next_cost:
                self.path[ny][nx] = Path(next_cost, step[2])
                return (ny,nx)
        return None
                      
    def update_path(self, node):
        return map(lambda s: self.update_step(node[0], node[1], s), Maze.steps)
        
    def update_values(self, nodes):
        s = set(filterfalse(lambda n: n is None or n == self.finish, chain.from_iterable(map(self.update_path, nodes))))
        return s
    
    def find(self):
        nodes = set( (self.start,) )
        while len(nodes) > 0:
            nodes = self.update_values(nodes)
        cur = self.finish
        res = []
        while cur[0] != self.start[0] or cur[1] != self.start[1]:
            d = self.path[cur[0]][cur[1]].dir
            res.append(d)
            if d == 'down':
                cur = (cur[0]-1, cur[1])
            elif d == 'up':
                cur = (cur[0]+1, cur[1])
            elif d == 'left':
                cur = (cur[0], cur[1]+1)
            elif d == 'right':
                cur = (cur[0], cur[1]-1)
        return res[::-1]

def find(toll_map, start, finish):
    m = Maze(toll_map, start, finish)
    return m.find()
        

In [1]:
# Version 2 - optimized
from collections import namedtuple, deque
from itertools import chain,filterfalse

class Maze(object):
                      
    steps = [ (0,1,1), (1,0,2), (-1,0,3), (0,-1,4) ] #1: 'right', 2: 'down', 3: 'up', 4: 'left'
    back_steps = {
        1: (0, -1, 'right'),
        2: (-1, 0, 'down'),
        3: (1,  0, 'up'),
        4: (0, 1, 'left')
    }
    MAX_VAL = float('Inf')
                      
    def __init__(self, toll_map, start, finish):
        self.toll_map = toll_map
        self.start = start
        self.finish = finish
        self.ysize = len(toll_map)
        self.xsize = len(toll_map[0])
    
    def is_valid(self, y, x):
        return y >= 0 and x >= 0 and y < self.ysize and x < self.xsize
    
    def find(self):
        nodes = (self.start,)
        path = [ [ (Maze.MAX_VAL, 0) ] * self.xsize for i in range(0, self.ysize) ]
        path[self.start[0]][self.start[1]] = (0, 0)
        finish_cost = Maze.MAX_VAL #make sure that points with value > current finish value will not be evaluated
        while len(nodes) > 0:
            old_nodes = nodes
            nodes = []
            for node in old_nodes:
                for step in Maze.steps:
                    ny, nx = next_node = ( node[0] + step[0], node[1] + step[1] )
                    if self.is_valid(ny, nx): 
                        next_cost = self.toll_map[ny][nx] + path[node[0]][node[1]][0]
                        if path[ny][nx][0] > next_cost:
                            path[ny][nx] = (next_cost, step[2])
                            if next_node == finish_cost:
                                finish_cost = min(next_cost, finish_cost)
                            elif next_cost < finish_cost:
                                nodes.append(next_node)
                                
        cur = self.finish
        res = []
        while cur != self.start:
            d = Maze.back_steps[ path[cur[0]][cur[1]][1] ] # get back step direction
            res.append(d[2])
            cur = (cur[0] + d[0], cur[1] + d[1])
        return res[::-1]

def find(toll_map, start, finish):
    m = Maze(toll_map, start, finish)
    return m.find()

In [2]:
# heapq version
from heapq import *

class MazeHQ(object):
                      
    steps = [ (0,1,1), (1,0,2), (-1,0,3), (0,-1,4) ] #1: 'right', 2: 'down', 3: 'up', 4: 'left'
    back_steps = {
        1: (0, -1, 'right'),
        2: (-1, 0, 'down'),
        3: (1,  0, 'up'),
        4: (0, 1, 'left')
    }
    MAX_VAL = float('Inf')
    
    def __init__(self, toll_map, start, finish):
        self.toll_map = toll_map
        self.start = start
        self.finish = finish
        self.ysize = len(toll_map)
        self.xsize = len(toll_map[0])
        
    def is_valid(self, y, x):
        return y >= 0 and x >= 0 and y < self.ysize and x < self.xsize
    
    def find(self):
        q = [(0, 0, self.start)]         # heapq - (cost, is_end, position) 
        path = { self.start: (0, 0) } # (cost, direction)
        
        while len(q) > 0 and q[0][-1] != self.finish:
            cur_node = heappop(q)
            y, x = cur_node[2]
            for step in steps:
                ny, nx = next_node = ( y + step[0], x + step[1] )
                next_cost = cur_node[0] + self.toll_map[y][x]
                if self.is_valid(ny, nx) or path.get(next_node, MazeHQ.MAX_VAL)[0] <= next_cost:
                    path[next_node] = (next_cost, step[2])
                    heappush(q, (next_cost, next_node == self.finish, next_node) )
                    
        cur = self.finish
        res = []
        while cur != self.start:
            d = Maze.back_steps[ path[cur][1] ] # get back step direction
            res.append(d[2])
            cur = (cur[0] + d[0], cur[1] + d[1])
        return res[::-1]

def findhq(toll_map, start, finish):
    m = Maze(toll_map, start, finish)
    return m.find()

In [3]:
toll_map = [  
    [1,9,1],  
    [2,9,1],  
    [2,1,1] ] 
find(toll_map, (0,0), (0,2))

['down', 'down', 'right', 'right', 'up', 'up']

In [4]:
toll_map = [  
    [1,9,1],  
    [2,9,1],  
    [2,1,1] ] 
findhq(toll_map, (0,0), (0,2))

['down', 'down', 'right', 'right', 'up', 'up']

In [5]:
toll_map = [  
    [1,4,1],  
    [1,9,1],  
    [1,1,1] ] 
find(toll_map, (0,0), (0,2))

['right', 'right']

In [6]:
toll_map = [  
    [1,4,1],  
    [1,9,1],  
    [1,1,1] ] 
findhq(toll_map, (0,0), (0,2))

['right', 'right']

In [7]:
toll_map = [
[38, 22, 21, 40, 51, 41, 49, 24, 42, 36, 66, 33, 93, 58, 70, 47, 88, 68, 53, 58, 16, 38, 74, 77, 18, 12, 90, 35, 17, 89, 81, 65, 51, 85, 69, 77, 5, 21, 68, 85, 37, 64, 51, 41, 38, 43, 87, 71, 63, 56, 59, 43, 43, 41, 20, 18, 97, 12, 68, 82, 86, 98, 27, 34, 9, 80, 2, 26, 12, 64, 96, 60, 23, 54, 27, 92, 30, 85, 20, 36, 2, 85, 17, 56, 7, 42, 50, 18, 29, 52, 18, 69, 8],
[2, 33, 1, 47, 49, 59, 90, 20, 8, 68, 87, 66, 2, 32, 10, 55, 71, 24, 67, 75, 71, 76, 3, 64, 23, 62, 29, 19, 30, 50, 1, 41, 8, 19, 10, 72, 89, 61, 42, 100, 39, 87, 35, 65, 26, 23, 1, 85, 9, 1, 39, 48, 50, 99, 37, 76, 75, 21, 4, 36, 38, 99, 54, 36, 69, 49, 98, 44, 93, 57, 24, 76, 39, 12, 100, 25, 98, 13, 82, 59, 40, 55, 64, 60, 94, 90, 79, 39, 7, 28, 87, 89, 57],
[32, 33, 71, 99, 40, 92, 72, 16, 62, 80, 54, 34, 47, 82, 43, 91, 100, 75, 73, 22, 84, 18, 93, 76, 64, 87, 80, 66, 82, 30, 16, 11, 45, 64, 16, 5, 23, 1, 81, 67, 13, 96, 91, 94, 88, 23, 17, 33, 97, 11, 16, 40, 54, 21, 21, 64, 68, 1, 79, 14, 54, 51, 72, 35, 70, 100, 40, 70, 15, 21, 41, 59, 2, 65, 72, 47, 36, 28, 58, 70, 88, 64, 93, 82, 46, 28, 25, 13, 72, 65, 17, 59, 9],
[80, 68, 36, 23, 93, 21, 90, 1, 46, 57, 26, 74, 52, 82, 99, 18, 79, 63, 7, 52, 71, 23, 75, 6, 91, 1, 69, 44, 69, 47, 5, 65, 12, 19, 65, 87, 13, 80, 85, 98, 15, 66, 68, 19, 61, 29, 64, 13, 100, 9, 71, 23, 79, 68, 7, 77, 52, 43, 67, 18, 36, 31, 33, 92, 90, 76, 1, 60, 26, 21, 24, 24, 24, 88, 96, 33, 45, 8, 88, 52, 68, 3, 77, 20, 32, 17, 73, 56, 89, 34, 1, 60, 73],
[40, 55, 46, 16, 18, 36, 7, 98, 22, 80, 31, 6, 97, 89, 91, 60, 60, 63, 6, 28, 98, 76, 77, 94, 29, 38, 11, 52, 5, 76, 66, 59, 96, 4, 26, 1, 74, 74, 47, 25, 45, 20, 37, 12, 60, 4, 92, 88, 92, 6, 14, 17, 40, 53, 98, 33, 50, 14, 18, 36, 71, 40, 36, 8, 64, 23, 16, 8, 24, 74, 57, 36, 7, 44, 49, 58, 29, 51, 51, 83, 40, 8, 66, 14, 97, 97, 79, 36, 4, 66, 45, 42, 42],
[100, 63, 25, 54, 35, 14, 12, 94, 96, 80, 79, 51, 16, 59, 81, 67, 57, 51, 71, 27, 33, 99, 93, 7, 31, 86, 29, 10, 91, 24, 98, 44, 41, 96, 67, 70, 63, 83, 66, 39, 51, 77, 20, 13, 76, 89, 55, 93, 19, 98, 51, 35, 78, 98, 21, 71, 57, 40, 89, 31, 22, 70, 31, 52, 58, 3, 6, 40, 46, 27, 36, 62, 90, 31, 75, 58, 1, 65, 17, 47, 51, 28, 54, 11, 66, 13, 83, 61, 59, 16, 60, 86, 19],
[99, 67, 92, 98, 34, 75, 22, 78, 2, 4, 77, 99, 35, 82, 63, 51, 36, 60, 75, 16, 70, 72, 13, 52, 88, 54, 76, 34, 13, 5, 40, 45, 60, 52, 16, 71, 9, 80, 40, 44, 19, 17, 34, 20, 16, 57, 50, 44, 40, 36, 27, 72, 89, 5, 45, 10, 86, 33, 86, 49, 25, 94, 5, 18, 31, 28, 71, 70, 31, 46, 85, 59, 78, 9, 30, 46, 27, 18, 85, 71, 68, 80, 4, 59, 84, 27, 2, 83, 76, 32, 11, 44, 35],
[31, 49, 32, 96, 84, 11, 19, 30, 18, 58, 60, 17, 4, 94, 2, 82, 3, 51, 25, 7, 55, 24, 81, 99, 40, 21, 67, 59, 98, 49, 84, 65, 50, 78, 53, 46, 30, 38, 91, 57, 29, 8, 77, 74, 12, 14, 70, 76, 72, 76, 86, 23, 38, 36, 82, 38, 49, 44, 13, 23, 75, 69, 58, 34, 18, 70, 5, 43, 98, 43, 64, 4, 62, 66, 66, 1, 34, 10, 40, 44, 86, 26, 43, 84, 97, 83, 5, 46, 20, 23, 46, 2, 30],
[64, 16, 14, 53, 79, 58, 62, 100, 84, 29, 28, 8, 95, 50, 26, 45, 4, 83, 31, 24, 92, 69, 77, 90, 98, 33, 17, 33, 86, 27, 5, 75, 4, 32, 93, 86, 51, 33, 13, 84, 76, 15, 96, 18, 71, 6, 68, 17, 29, 83, 94, 55, 23, 99, 24, 61, 84, 55, 7, 70, 4, 54, 22, 55, 49, 11, 67, 11, 89, 82, 67, 97, 92, 61, 52, 19, 35, 27, 96, 27, 42, 69, 4, 68, 97, 10, 73, 47, 95, 19, 27, 59, 44],
[29, 69, 66, 72, 74, 98, 4, 14, 4, 4, 79, 44, 83, 55, 78, 3, 21, 12, 51, 12, 53, 80, 56, 85, 66, 89, 51, 87, 46, 98, 67, 82, 38, 93, 61, 84, 82, 49, 96, 2, 71, 60, 42, 32, 68, 84, 97, 2, 62, 71, 34, 8, 6, 89, 44, 8, 3, 57, 51, 17, 27, 62, 94, 72, 33, 69, 92, 75, 75, 3, 80, 75, 24, 26, 61, 84, 29, 5, 82, 91, 25, 40, 77, 24, 84, 96, 12, 14, 83, 45, 87, 40, 8],
[29, 16, 12, 63, 73, 51, 74, 89, 72, 27, 37, 14, 47, 57, 3, 4, 31, 2, 4, 35, 52, 1, 98, 95, 16, 99, 56, 33, 96, 21, 10, 30, 34, 90, 56, 55, 85, 43, 42, 76, 83, 34, 41, 48, 13, 18, 22, 91, 34, 62, 54, 86, 59, 54, 23, 94, 51, 68, 84, 25, 28, 58, 14, 79, 66, 59, 31, 19, 57, 2, 75, 96, 83, 32, 1, 11, 91, 37, 35, 50, 20, 61, 56, 62, 18, 39, 9, 42, 43, 27, 29, 62, 53],
[34, 15, 12, 31, 49, 88, 66, 2, 90, 62, 7, 66, 73, 35, 2, 96, 26, 77, 30, 83, 67, 67, 24, 33, 95, 17, 18, 52, 1, 4, 61, 99, 3, 16, 28, 96, 21, 28, 90, 25, 86, 95, 91, 97, 26, 89, 35, 42, 51, 26, 5, 88, 27, 49, 16, 77, 47, 45, 53, 3, 2, 10, 37, 7, 21, 77, 100, 64, 88, 61, 15, 97, 46, 13, 77, 62, 26, 53, 59, 29, 77, 44, 47, 82, 78, 34, 13, 68, 40, 22, 77, 62, 21],
[77, 14, 73, 33, 49, 50, 63, 67, 57, 32, 52, 74, 94, 4, 7, 14, 38, 25, 62, 28, 89, 20, 50, 61, 15, 59, 77, 78, 16, 70, 61, 62, 61, 2, 86, 44, 82, 11, 10, 76, 94, 91, 7, 39, 43, 63, 33, 52, 43, 10, 25, 3, 28, 18, 53, 27, 39, 16, 9, 67, 90, 46, 88, 42, 58, 88, 5, 3, 41, 12, 87, 3, 56, 99, 84, 81, 99, 49, 40, 97, 62, 20, 67, 7, 49, 74, 43, 100, 39, 78, 88, 64, 15],
[76, 90, 98, 53, 33, 18, 25, 91, 40, 15, 80, 39, 21, 27, 38, 69, 53, 65, 12, 28, 19, 18, 97, 78, 28, 66, 77, 88, 22, 87, 54, 57, 70, 3, 90, 81, 75, 52, 84, 21, 22, 22, 86, 93, 97, 23, 23, 20, 46, 74, 52, 26, 47, 4, 82, 7, 60, 42, 13, 50, 62, 3, 11, 9, 80, 60, 4, 91, 85, 21, 46, 11, 87, 21, 9, 21, 19, 15, 80, 35, 16, 30, 29, 18, 58, 66, 24, 71, 8, 80, 41, 53, 77],
[18, 98, 31, 58, 92, 3, 97, 61, 10, 61, 25, 67, 57, 5, 49, 87, 40, 69, 3, 100, 40, 6, 32, 45, 98, 29, 10, 33, 42, 58, 15, 87, 48, 26, 62, 42, 3, 29, 47, 29, 84, 38, 83, 95, 45, 81, 14, 82, 73, 83, 79, 76, 13, 66, 58, 96, 37, 29, 86, 26, 43, 15, 60, 11, 74, 93, 3, 80, 83, 73, 28, 57, 72, 72, 12, 82, 30, 16, 46, 18, 96, 100, 28, 15, 77, 27, 12, 26, 64, 60, 85, 8, 12],
[62, 33, 52, 35, 89, 48, 47, 53, 14, 95, 13, 29, 32, 98, 1, 11, 17, 71, 70, 14, 91, 36, 80, 48, 6, 97, 78, 10, 43, 18, 86, 45, 96, 24, 89, 60, 9, 52, 14, 91, 29, 92, 76, 16, 92, 86, 27, 77, 76, 19, 48, 63, 90, 51, 58, 2, 17, 11, 48, 47, 68, 52, 32, 83, 97, 36, 80, 27, 50, 2, 15, 66, 14, 99, 50, 43, 78, 24, 85, 80, 65, 3, 71, 75, 56, 49, 66, 41, 39, 84, 18, 29, 43],
[39, 90, 93, 49, 2, 84, 36, 4, 60, 4, 69, 25, 48, 65, 5, 23, 90, 52, 41, 37, 58, 99, 44, 56, 48, 10, 24, 89, 32, 48, 19, 56, 56, 42, 5, 47, 35, 85, 65, 14, 28, 92, 85, 71, 74, 3, 60, 89, 78, 57, 96, 25, 94, 67, 71, 7, 43, 22, 28, 25, 38, 5, 20, 79, 14, 33, 39, 90, 64, 94, 75, 51, 35, 34, 19, 38, 57, 82, 49, 4, 93, 2, 82, 34, 46, 42, 22, 61, 85, 75, 35, 64, 5],
[16, 1, 20, 1, 92, 14, 9, 92, 84, 71, 19, 4, 12, 74, 55, 51, 74, 43, 93, 44, 66, 52, 49, 84, 78, 46, 44, 70, 51, 41, 15, 37, 18, 79, 81, 10, 21, 70, 44, 61, 50, 7, 82, 49, 64, 24, 44, 51, 7, 82, 26, 32, 9, 86, 29, 52, 16, 43, 90, 11, 19, 87, 4, 49, 32, 37, 81, 22, 71, 98, 71, 81, 96, 52, 55, 100, 73, 87, 85, 45, 2, 83, 37, 50, 47, 57, 26, 74, 100, 92, 4, 68, 4],
[96, 14, 37, 22, 30, 30, 23, 38, 78, 9, 71, 83, 72, 87, 58, 18, 68, 96, 78, 60, 86, 68, 8, 62, 28, 90, 97, 35, 86, 4, 68, 40, 4, 48, 48, 38, 48, 95, 43, 27, 40, 38, 40, 77, 72, 22, 63, 85, 13, 38, 42, 24, 68, 31, 97, 23, 22, 57, 98, 19, 9, 76, 29, 97, 63, 87, 30, 29, 97, 37, 6, 84, 52, 27, 82, 53, 9, 79, 47, 59, 29, 45, 20, 57, 90, 63, 65, 11, 62, 36, 70, 5, 95],
[16, 60, 1, 11, 96, 30, 93, 85, 58, 49, 94, 93, 17, 26, 37, 38, 31, 51, 92, 1, 98, 30, 46, 41, 88, 50, 13, 10, 75, 87, 78, 53, 87, 78, 84, 55, 2, 8, 25, 57, 16, 91, 73, 64, 82, 79, 68, 98, 31, 25, 75, 93, 6, 22, 88, 85, 24, 87, 71, 20, 18, 94, 4, 93, 3, 25, 2, 98, 6, 86, 19, 64, 30, 47, 28, 78, 93, 1, 6, 89, 77, 100, 81, 3, 17, 77, 17, 95, 94, 49, 75, 38, 28],
[27, 96, 2, 3, 61, 72, 81, 18, 88, 53, 11, 10, 93, 16, 7, 71, 91, 63, 75, 3, 61, 67, 42, 98, 7, 8, 5, 96, 68, 72, 58, 99, 26, 85, 34, 90, 74, 60, 64, 61, 77, 66, 85, 100, 83, 75, 15, 40, 10, 36, 69, 66, 13, 37, 42, 18, 32, 65, 10, 100, 52, 52, 78, 64, 11, 49, 53, 14, 10, 38, 17, 61, 55, 41, 22, 45, 32, 31, 92, 88, 68, 44, 98, 42, 37, 19, 88, 41, 56, 16, 16, 82, 63],
[97, 74, 30, 78, 35, 26, 2, 69, 2, 75, 14, 79, 61, 66, 22, 21, 67, 7, 8, 85, 93, 81, 51, 10, 18, 28, 27, 24, 62, 18, 5, 63, 73, 67, 97, 73, 84, 50, 52, 29, 32, 36, 3, 25, 88, 59, 47, 86, 82, 18, 2, 25, 55, 39, 84, 23, 9, 87, 93, 19, 35, 15, 40, 74, 13, 34, 46, 76, 79, 68, 16, 41, 28, 42, 68, 84, 74, 66, 52, 98, 25, 99, 35, 39, 22, 90, 80, 68, 4, 5, 46, 93, 17],
[20, 71, 88, 100, 91, 63, 97, 99, 66, 36, 17, 44, 13, 65, 40, 11, 100, 9, 36, 93, 78, 35, 29, 58, 60, 20, 99, 63, 47, 44, 3, 45, 88, 7, 62, 92, 22, 90, 34, 53, 15, 7, 85, 29, 38, 37, 76, 1, 92, 65, 40, 69, 9, 69, 52, 55, 70, 6, 82, 30, 39, 33, 96, 34, 48, 45, 82, 52, 75, 55, 84, 79, 77, 42, 60, 74, 9, 95, 85, 82, 17, 63, 3, 100, 65, 77, 44, 69, 32, 96, 39, 61, 78],
[46, 35, 72, 20, 94, 46, 53, 79, 57, 34, 10, 29, 35, 73, 99, 74, 54, 80, 43, 94, 65, 50, 36, 89, 15, 80, 62, 56, 29, 13, 31, 15, 82, 6, 41, 76, 100, 84, 92, 89, 32, 25, 57, 38, 77, 27, 67, 58, 89, 24, 4, 61, 93, 76, 19, 34, 55, 8, 15, 53, 26, 83, 50, 73, 66, 60, 30, 72, 92, 2, 18, 23, 45, 38, 17, 96, 36, 3, 67, 54, 98, 63, 43, 39, 70, 59, 7, 30, 42, 15, 52, 10, 38],
[62, 24, 29, 25, 62, 64, 82, 3, 28, 4, 46, 7, 84, 7, 84, 44, 2, 31, 83, 39, 70, 54, 15, 31, 35, 30, 57, 65, 73, 2, 51, 49, 21, 5, 26, 43, 69, 20, 44, 63, 90, 62, 22, 29, 91, 91, 14, 94, 15, 75, 55, 30, 99, 31, 25, 15, 37, 13, 8, 65, 72, 2, 81, 51, 30, 35, 40, 17, 95, 56, 50, 10, 68, 65, 80, 12, 38, 36, 29, 23, 95, 69, 51, 92, 43, 80, 24, 21, 29, 8, 10, 6, 93],
[32, 1, 76, 41, 13, 27, 32, 33, 1, 4, 65, 39, 19, 94, 50, 38, 33, 46, 75, 8, 38, 10, 55, 13, 18, 21, 75, 24, 44, 14, 94, 70, 79, 37, 42, 92, 63, 47, 31, 82, 22, 93, 8, 89, 57, 73, 11, 66, 41, 86, 50, 72, 25, 64, 84, 79, 100, 43, 11, 12, 96, 86, 53, 61, 75, 72, 25, 49, 95, 17, 100, 91, 35, 49, 6, 48, 18, 19, 75, 89, 1, 48, 69, 16, 69, 60, 59, 56, 1, 67, 44, 15, 97],
[88, 40, 90, 97, 95, 74, 26, 41, 96, 9, 95, 55, 5, 78, 67, 80, 19, 96, 2, 89, 48, 89, 58, 39, 82, 15, 61, 81, 33, 90, 46, 43, 95, 27, 11, 2, 53, 43, 59, 9, 5, 33, 44, 25, 22, 8, 76, 63, 68, 37, 52, 13, 73, 83, 34, 94, 72, 87, 40, 24, 73, 70, 75, 93, 33, 78, 34, 72, 61, 64, 29, 74, 82, 41, 87, 58, 42, 56, 75, 22, 40, 58, 84, 70, 13, 17, 69, 72, 34, 73, 72, 98, 69],
[9, 48, 100, 96, 10, 3, 18, 48, 63, 14, 64, 24, 7, 46, 6, 52, 74, 33, 19, 9, 35, 65, 13, 70, 60, 75, 62, 55, 75, 14, 18, 50, 39, 61, 90, 81, 7, 72, 52, 33, 13, 57, 57, 12, 60, 59, 37, 33, 96, 23, 93, 92, 41, 46, 59, 82, 3, 72, 64, 18, 77, 67, 16, 71, 66, 70, 12, 67, 67, 39, 76, 37, 23, 92, 71, 41, 10, 78, 41, 73, 8, 3, 28, 41, 55, 70, 29, 73, 56, 48, 95, 32, 31],
[99, 72, 25, 49, 9, 96, 62, 99, 11, 63, 72, 6, 39, 98, 64, 47, 40, 41, 56, 12, 69, 13, 63, 19, 30, 18, 79, 22, 63, 68, 70, 84, 21, 37, 5, 87, 31, 17, 2, 53, 62, 63, 83, 26, 26, 84, 29, 35, 96, 87, 5, 28, 86, 3, 79, 69, 15, 64, 64, 4, 81, 97, 27, 33, 100, 46, 38, 63, 51, 33, 35, 1, 31, 25, 97, 73, 5, 73, 66, 56, 97, 63, 8, 84, 14, 67, 62, 81, 82, 37, 13, 46, 8],
[73, 93, 18, 15, 30, 57, 38, 46, 44, 79, 54, 60, 36, 85, 90, 91, 93, 60, 1, 35, 50, 87, 93, 55, 54, 74, 32, 10, 10, 19, 60, 80, 41, 64, 91, 32, 51, 50, 22, 17, 64, 48, 66, 45, 74, 40, 79, 95, 24, 93, 6, 17, 2, 21, 38, 69, 19, 18, 16, 24, 62, 83, 7, 56, 48, 70, 41, 35, 98, 17, 18, 19, 77, 13, 14, 94, 95, 94, 74, 25, 84, 21, 51, 4, 16, 38, 18, 30, 6, 98, 79, 67, 98],
[18, 30, 89, 99, 77, 22, 2, 27, 23, 12, 61, 5, 40, 11, 49, 16, 8, 24, 35, 2, 60, 6, 53, 30, 68, 93, 97, 4, 89, 91, 26, 96, 1, 41, 43, 74, 10, 33, 85, 97, 60, 76, 74, 17, 42, 77, 56, 61, 51, 28, 90, 78, 81, 81, 9, 65, 55, 50, 15, 4, 14, 75, 67, 12, 78, 16, 63, 12, 37, 41, 66, 28, 5, 48, 37, 64, 3, 39, 55, 68, 57, 94, 17, 9, 97, 93, 68, 11, 80, 31, 17, 48, 64],
[71, 48, 80, 74, 54, 34, 25, 39, 7, 85, 6, 100, 11, 70, 68, 51, 4, 26, 95, 41, 34, 56, 33, 89, 7, 46, 19, 92, 34, 28, 39, 99, 54, 41, 92, 70, 86, 12, 43, 36, 8, 86, 29, 21, 40, 55, 36, 92, 93, 25, 13, 22, 45, 14, 91, 3, 50, 36, 90, 15, 75, 55, 67, 94, 14, 15, 30, 77, 19, 96, 100, 53, 31, 52, 45, 100, 2, 40, 61, 91, 56, 68, 80, 18, 54, 4, 84, 88, 3, 15, 46, 64, 88],
[10, 49, 93, 90, 24, 93, 39, 73, 22, 74, 16, 42, 75, 95, 19, 82, 9, 6, 6, 22, 85, 60, 37, 34, 53, 83, 67, 44, 71, 30, 80, 28, 74, 41, 36, 33, 69, 53, 17, 13, 62, 55, 43, 86, 73, 50, 27, 85, 42, 20, 32, 57, 90, 33, 64, 90, 34, 69, 68, 65, 25, 10, 71, 91, 21, 37, 14, 55, 60, 27, 67, 15, 36, 100, 37, 10, 61, 76, 35, 35, 10, 84, 17, 10, 96, 22, 99, 17, 16, 53, 36, 13, 80],
[56, 55, 30, 50, 6, 100, 64, 84, 61, 38, 40, 65, 93, 46, 75, 95, 37, 15, 46, 2, 2, 99, 72, 89, 94, 89, 55, 74, 76, 60, 96, 36, 3, 77, 54, 24, 82, 49, 92, 84, 67, 83, 38, 42, 86, 90, 12, 3, 80, 19, 62, 96, 38, 38, 27, 61, 69, 3, 86, 96, 97, 19, 19, 74, 29, 39, 92, 41, 84, 51, 89, 52, 10, 51, 64, 90, 2, 77, 82, 95, 38, 32, 82, 70, 96, 91, 20, 2, 1, 44, 40, 43, 91],
[4, 59, 44, 63, 29, 18, 36, 31, 45, 80, 84, 23, 2, 31, 70, 37, 79, 86, 45, 3, 34, 2, 37, 9, 74, 97, 60, 49, 98, 43, 13, 53, 21, 69, 31, 77, 26, 65, 15, 68, 67, 65, 43, 92, 98, 40, 29, 30, 49, 65, 91, 13, 78, 57, 70, 99, 41, 49, 38, 72, 9, 4, 70, 6, 1, 56, 6, 66, 4, 56, 54, 66, 92, 78, 59, 96, 32, 92, 28, 48, 28, 82, 72, 46, 35, 26, 78, 56, 92, 42, 100, 75, 65],
[28, 81, 5, 53, 16, 22, 38, 28, 98, 70, 24, 86, 88, 83, 31, 45, 35, 32, 34, 48, 77, 94, 38, 79, 28, 34, 69, 98, 36, 44, 29, 18, 79, 81, 27, 8, 36, 28, 4, 72, 19, 9, 62, 6, 30, 96, 5, 22, 36, 31, 15, 86, 35, 100, 85, 12, 12, 24, 94, 87, 85, 65, 82, 83, 1, 34, 52, 71, 42, 57, 66, 73, 58, 59, 58, 67, 99, 30, 16, 12, 15, 48, 21, 99, 48, 41, 41, 65, 83, 23, 88, 4, 29],
[66, 42, 42, 27, 28, 72, 91, 55, 31, 38, 50, 77, 38, 89, 33, 20, 54, 4, 8, 79, 95, 56, 4, 74, 75, 82, 67, 6, 68, 56, 74, 5, 89, 7, 46, 2, 50, 13, 35, 42, 51, 85, 53, 73, 67, 50, 28, 16, 19, 36, 2, 100, 72, 6, 88, 75, 14, 27, 27, 75, 35, 66, 64, 80, 92, 81, 21, 31, 69, 80, 14, 86, 4, 38, 72, 49, 1, 73, 13, 73, 10, 81, 53, 9, 100, 40, 82, 44, 67, 33, 42, 82, 35],
[59, 54, 75, 43, 47, 77, 21, 80, 20, 54, 23, 56, 45, 12, 76, 24, 18, 35, 38, 43, 20, 16, 51, 48, 19, 50, 89, 37, 44, 42, 98, 17, 48, 11, 53, 80, 3, 79, 75, 17, 71, 19, 35, 76, 5, 56, 73, 7, 32, 97, 60, 28, 34, 62, 35, 91, 89, 42, 61, 62, 39, 57, 91, 8, 70, 88, 73, 72, 21, 63, 100, 90, 57, 96, 18, 66, 16, 44, 76, 93, 51, 76, 90, 45, 60, 45, 53, 93, 47, 29, 23, 13, 11],
[87, 82, 76, 71, 91, 66, 54, 52, 59, 50, 11, 24, 75, 93, 29, 47, 70, 70, 83, 25, 84, 40, 22, 75, 15, 52, 76, 100, 11, 7, 11, 87, 51, 93, 71, 29, 99, 33, 49, 8, 21, 67, 2, 20, 58, 11, 32, 5, 2, 81, 31, 83, 20, 22, 67, 71, 24, 90, 99, 42, 4, 54, 10, 18, 93, 63, 69, 24, 59, 73, 86, 3, 39, 14, 76, 22, 99, 21, 37, 49, 13, 9, 70, 51, 62, 85, 26, 34, 37, 100, 87, 26, 94],
[92, 78, 4, 46, 90, 33, 49, 69, 18, 24, 94, 34, 34, 30, 76, 44, 84, 26, 27, 45, 67, 51, 54, 54, 82, 20, 17, 97, 97, 19, 26, 21, 36, 87, 22, 90, 88, 97, 22, 82, 41, 100, 42, 13, 3, 20, 10, 77, 74, 64, 81, 90, 60, 42, 27, 61, 77, 92, 69, 79, 6, 57, 60, 65, 57, 8, 86, 86, 28, 64, 6, 89, 32, 21, 5, 46, 24, 75, 28, 74, 54, 67, 10, 51, 85, 24, 30, 85, 15, 65, 6, 74, 12],
[5, 57, 3, 58, 22, 54, 12, 95, 18, 49, 35, 87, 62, 20, 21, 71, 44, 35, 27, 77, 19, 1, 17, 27, 45, 12, 41, 50, 66, 67, 64, 13, 16, 88, 1, 75, 84, 32, 7, 25, 64, 15, 46, 97, 82, 12, 59, 37, 35, 30, 81, 54, 31, 90, 8, 90, 89, 72, 95, 85, 45, 93, 3, 23, 87, 29, 85, 19, 52, 10, 43, 6, 43, 49, 73, 79, 4, 5, 31, 42, 48, 52, 66, 53, 24, 56, 62, 61, 11, 9, 2, 94, 54],
[48, 25, 53, 54, 76, 43, 93, 32, 53, 23, 11, 36, 6, 89, 96, 18, 4, 83, 72, 53, 74, 86, 79, 47, 79, 8, 55, 10, 1, 88, 51, 58, 30, 13, 99, 93, 35, 33, 87, 18, 28, 75, 6, 82, 13, 57, 84, 64, 11, 38, 75, 9, 33, 85, 76, 75, 86, 96, 85, 59, 37, 51, 98, 13, 85, 99, 29, 6, 98, 2, 89, 70, 86, 96, 38, 17, 75, 73, 73, 42, 25, 77, 43, 22, 80, 88, 99, 15, 13, 32, 59, 21, 85],
[51, 21, 38, 46, 68, 88, 83, 45, 24, 64, 56, 93, 37, 35, 3, 60, 10, 79, 75, 61, 67, 35, 74, 18, 60, 60, 12, 8, 77, 40, 22, 86, 9, 53, 30, 15, 23, 73, 52, 87, 78, 45, 40, 9, 80, 60, 94, 68, 41, 34, 24, 65, 47, 16, 56, 70, 56, 34, 91, 92, 41, 81, 45, 92, 19, 45, 45, 60, 36, 96, 69, 68, 57, 19, 24, 20, 38, 67, 90, 96, 8, 5, 94, 38, 5, 90, 9, 31, 62, 73, 23, 48, 21],
[20, 76, 36, 77, 64, 4, 86, 8, 63, 93, 75, 64, 52, 88, 23, 24, 40, 53, 87, 12, 6, 16, 1, 1, 42, 89, 33, 4, 74, 21, 26, 76, 51, 61, 85, 63, 96, 8, 71, 66, 66, 7, 61, 8, 70, 81, 54, 9, 62, 52, 13, 49, 6, 62, 96, 18, 49, 71, 99, 93, 51, 54, 2, 28, 80, 23, 51, 100, 61, 99, 25, 2, 28, 34, 59, 73, 55, 25, 80, 39, 42, 93, 67, 50, 74, 15, 89, 12, 34, 71, 99, 55, 35],
[3, 88, 77, 58, 14, 50, 31, 20, 67, 82, 83, 50, 72, 39, 54, 88, 92, 82, 81, 30, 54, 63, 91, 36, 19, 22, 86, 19, 24, 87, 66, 60, 60, 70, 18, 14, 37, 90, 99, 99, 26, 47, 33, 73, 83, 48, 46, 21, 45, 9, 48, 67, 2, 8, 27, 28, 2, 6, 8, 61, 19, 82, 8, 87, 29, 59, 34, 44, 96, 92, 95, 77, 47, 65, 29, 30, 94, 5, 10, 32, 15, 6, 11, 25, 71, 10, 46, 16, 32, 71, 97, 51, 53],
[16, 20, 32, 81, 26, 33, 12, 77, 94, 42, 28, 73, 16, 66, 16, 2, 71, 77, 100, 65, 67, 32, 61, 75, 34, 3, 60, 38, 98, 12, 76, 51, 22, 17, 76, 60, 82, 78, 2, 23, 51, 3, 80, 31, 20, 54, 7, 23, 1, 16, 47, 50, 86, 92, 22, 99, 32, 94, 40, 77, 70, 30, 56, 8, 59, 32, 21, 19, 99, 19, 38, 92, 30, 56, 25, 85, 5, 62, 47, 92, 79, 59, 91, 19, 13, 64, 48, 9, 39, 67, 46, 34, 43],
[64, 16, 12, 72, 93, 6, 47, 4, 81, 70, 52, 8, 68, 24, 8, 76, 1, 9, 23, 34, 70, 92, 43, 82, 60, 56, 82, 13, 81, 49, 71, 95, 31, 85, 3, 84, 84, 43, 76, 41, 29, 26, 4, 89, 19, 46, 95, 74, 22, 96, 7, 53, 16, 18, 41, 37, 81, 15, 74, 74, 67, 51, 14, 24, 17, 4, 5, 98, 24, 12, 82, 34, 67, 35, 57, 9, 77, 100, 79, 17, 26, 30, 15, 7, 97, 7, 69, 20, 59, 44, 100, 56, 37],
[28, 23, 44, 34, 14, 14, 96, 7, 13, 64, 53, 93, 17, 82, 21, 42, 13, 84, 64, 55, 54, 64, 8, 64, 46, 66, 51, 11, 35, 18, 28, 1, 45, 14, 14, 37, 49, 38, 37, 60, 54, 91, 84, 93, 63, 91, 18, 88, 2, 50, 9, 56, 83, 90, 98, 86, 40, 46, 20, 50, 77, 100, 53, 50, 95, 51, 2, 98, 78, 46, 95, 64, 17, 36, 10, 76, 72, 71, 39, 85, 51, 16, 58, 46, 96, 98, 99, 44, 59, 18, 65, 33, 41],
[25, 90, 43, 36, 5, 82, 18, 56, 80, 34, 48, 18, 79, 38, 57, 36, 36, 2, 56, 28, 60, 28, 28, 71, 100, 64, 25, 18, 95, 45, 33, 75, 72, 80, 25, 87, 51, 89, 77, 16, 91, 41, 84, 98, 90, 53, 6, 55, 47, 74, 54, 93, 34, 86, 4, 88, 83, 47, 15, 88, 81, 29, 82, 34, 19, 14, 94, 54, 6, 69, 95, 73, 1, 96, 70, 18, 48, 98, 32, 62, 88, 53, 89, 17, 25, 80, 53, 46, 19, 97, 42, 34, 95],
[85, 39, 89, 39, 19, 50, 63, 94, 69, 36, 64, 40, 41, 41, 65, 69, 53, 56, 23, 64, 3, 82, 33, 82, 95, 53, 80, 38, 16, 64, 94, 28, 6, 94, 38, 2, 63, 16, 44, 16, 56, 31, 19, 18, 79, 13, 58, 59, 81, 62, 6, 26, 66, 24, 11, 54, 93, 39, 30, 16, 44, 18, 20, 36, 65, 17, 22, 62, 28, 54, 59, 22, 28, 4, 31, 3, 5, 35, 26, 5, 21, 34, 91, 76, 70, 25, 64, 40, 56, 69, 6, 22, 27],
[32, 56, 82, 48, 13, 15, 7, 28, 61, 80, 89, 47, 64, 31, 66, 7, 3, 73, 46, 93, 79, 10, 11, 80, 13, 41, 51, 44, 78, 62, 3, 35, 24, 99, 71, 90, 22, 54, 75, 31, 48, 22, 29, 58, 20, 90, 16, 44, 58, 40, 44, 13, 8, 97, 70, 31, 15, 14, 58, 94, 26, 11, 38, 87, 35, 32, 40, 20, 24, 83, 48, 60, 4, 41, 36, 27, 39, 38, 23, 89, 6, 62, 76, 20, 73, 47, 77, 99, 79, 18, 48, 78, 82],
[59, 38, 41, 78, 74, 88, 21, 33, 81, 43, 70, 81, 55, 50, 51, 93, 28, 1, 44, 94, 54, 8, 67, 83, 30, 14, 22, 61, 75, 67, 95, 19, 58, 92, 34, 6, 20, 57, 8, 48, 94, 75, 27, 1, 25, 82, 66, 13, 29, 58, 49, 24, 29, 27, 58, 33, 5, 50, 49, 27, 31, 18, 6, 99, 91, 37, 52, 90, 96, 3, 42, 71, 18, 42, 73, 49, 59, 61, 23, 18, 43, 28, 98, 54, 94, 73, 70, 76, 20, 74, 6, 21, 22],
[19, 6, 83, 8, 16, 84, 55, 10, 81, 3, 65, 98, 48, 5, 80, 41, 4, 66, 49, 73, 72, 95, 53, 94, 20, 73, 18, 64, 92, 9, 29, 43, 32, 75, 67, 22, 12, 25, 2, 59, 24, 74, 1, 26, 86, 64, 2, 40, 26, 7, 1, 71, 96, 60, 57, 8, 22, 60, 99, 28, 51, 64, 77, 88, 13, 74, 39, 13, 55, 30, 53, 36, 67, 40, 14, 32, 95, 81, 85, 84, 46, 49, 55, 30, 3, 78, 24, 57, 94, 25, 12, 24, 57],
[39, 62, 76, 57, 38, 88, 30, 96, 90, 6, 28, 42, 66, 85, 39, 21, 76, 12, 84, 26, 89, 52, 38, 90, 93, 89, 58, 100, 49, 37, 41, 74, 7, 25, 12, 29, 67, 28, 64, 20, 52, 44, 3, 1, 71, 62, 12, 5, 77, 38, 42, 57, 21, 56, 86, 60, 97, 95, 15, 42, 36, 22, 53, 48, 94, 6, 94, 50, 75, 47, 24, 1, 100, 73, 27, 55, 35, 91, 7, 76, 10, 100, 33, 92, 39, 3, 68, 67, 51, 90, 48, 96, 97],
[78, 91, 50, 84, 63, 43, 39, 9, 93, 99, 76, 28, 74, 91, 70, 41, 26, 3, 61, 79, 25, 49, 82, 33, 74, 57, 46, 85, 89, 35, 25, 31, 83, 70, 53, 1, 39, 21, 88, 36, 76, 89, 98, 83, 94, 4, 46, 93, 85, 72, 60, 55, 47, 37, 1, 30, 5, 68, 6, 21, 45, 88, 79, 55, 87, 94, 12, 61, 77, 20, 46, 18, 46, 46, 73, 46, 37, 81, 58, 47, 69, 8, 50, 69, 91, 21, 98, 37, 14, 43, 12, 38, 30],
[79, 33, 5, 83, 66, 77, 46, 47, 79, 73, 99, 27, 2, 35, 30, 89, 11, 39, 46, 21, 7, 17, 89, 47, 51, 64, 75, 70, 13, 97, 30, 14, 90, 79, 81, 54, 28, 48, 4, 73, 84, 14, 63, 18, 54, 99, 16, 78, 48, 17, 16, 36, 9, 47, 34, 3, 56, 57, 67, 80, 96, 57, 10, 15, 87, 97, 6, 26, 42, 58, 52, 29, 92, 9, 22, 100, 36, 16, 78, 69, 53, 10, 68, 48, 57, 26, 87, 37, 49, 87, 96, 41, 97],
[48, 50, 79, 92, 93, 24, 15, 95, 65, 19, 33, 94, 80, 27, 70, 68, 91, 51, 16, 91, 19, 76, 77, 56, 7, 1, 59, 99, 95, 13, 48, 30, 45, 80, 21, 53, 26, 42, 9, 40, 9, 76, 2, 51, 38, 91, 11, 94, 99, 61, 13, 41, 79, 65, 17, 54, 5, 17, 20, 58, 24, 20, 79, 73, 5, 1, 76, 31, 82, 40, 91, 30, 95, 69, 42, 79, 41, 91, 17, 72, 41, 97, 79, 5, 100, 63, 81, 5, 51, 46, 97, 4, 69],
[38, 28, 41, 84, 45, 86, 23, 3, 60, 82, 65, 84, 47, 61, 15, 53, 35, 33, 14, 94, 73, 74, 25, 80, 71, 27, 62, 45, 14, 30, 78, 26, 4, 32, 16, 62, 94, 48, 88, 54, 57, 91, 79, 84, 43, 21, 72, 67, 93, 39, 78, 100, 25, 57, 67, 47, 69, 52, 15, 77, 64, 15, 86, 69, 1, 61, 48, 1, 8, 2, 39, 53, 93, 46, 64, 24, 31, 54, 93, 77, 12, 13, 55, 53, 72, 21, 53, 28, 61, 34, 82, 83, 58],
[83, 37, 76, 59, 39, 64, 55, 68, 62, 23, 64, 41, 73, 67, 51, 36, 5, 46, 15, 59, 33, 28, 35, 21, 13, 20, 69, 76, 4, 45, 70, 32, 41, 2, 60, 30, 58, 62, 10, 12, 28, 13, 70, 18, 65, 17, 98, 56, 22, 54, 20, 78, 26, 3, 51, 100, 44, 70, 13, 55, 4, 4, 75, 39, 13, 52, 23, 37, 43, 74, 43, 75, 98, 28, 41, 55, 48, 74, 50, 54, 9, 57, 16, 36, 15, 89, 82, 22, 21, 84, 72, 34, 7],
[26, 27, 96, 9, 97, 55, 33, 33, 87, 12, 49, 38, 91, 46, 39, 24, 47, 77, 78, 51, 5, 45, 21, 94, 9, 34, 70, 46, 15, 87, 67, 75, 7, 10, 39, 32, 36, 77, 74, 99, 43, 79, 77, 77, 37, 97, 42, 89, 59, 92, 97, 29, 94, 6, 61, 24, 62, 62, 78, 6, 2, 4, 36, 92, 2, 20, 28, 98, 46, 7, 64, 43, 63, 66, 19, 9, 97, 87, 72, 56, 89, 35, 20, 36, 79, 58, 74, 88, 10, 79, 31, 5, 14],
[27, 66, 51, 97, 25, 25, 38, 36, 42, 88, 85, 53, 25, 34, 73, 60, 9, 64, 82, 88, 47, 86, 55, 81, 57, 85, 89, 41, 78, 90, 74, 86, 44, 55, 39, 84, 6, 73, 89, 99, 17, 60, 46, 25, 91, 57, 66, 28, 76, 18, 71, 45, 34, 69, 54, 21, 96, 33, 32, 63, 5, 44, 73, 72, 93, 33, 11, 14, 18, 45, 69, 53, 80, 1, 100, 27, 11, 28, 37, 100, 69, 80, 91, 13, 37, 17, 56, 21, 86, 63, 69, 22, 60],
[77, 64, 46, 56, 83, 71, 1, 68, 34, 12, 60, 68, 40, 32, 92, 96, 90, 72, 46, 43, 37, 18, 80, 59, 66, 51, 59, 97, 41, 29, 77, 99, 66, 60, 73, 80, 8, 14, 62, 15, 20, 94, 61, 3, 49, 40, 72, 21, 45, 5, 54, 69, 44, 32, 51, 58, 4, 14, 11, 28, 46, 67, 57, 80, 41, 2, 5, 33, 88, 36, 11, 20, 14, 46, 99, 14, 57, 63, 19, 4, 100, 48, 100, 24, 56, 55, 33, 51, 69, 22, 60, 80, 18],
[95, 26, 30, 94, 5, 44, 95, 68, 73, 87, 69, 34, 11, 62, 16, 27, 44, 24, 2, 77, 57, 62, 35, 18, 32, 47, 86, 6, 61, 87, 75, 43, 33, 51, 66, 95, 90, 13, 36, 27, 13, 69, 62, 5, 6, 9, 87, 65, 14, 47, 64, 27, 42, 68, 63, 2, 72, 54, 91, 90, 11, 60, 78, 82, 9, 55, 69, 44, 79, 63, 14, 24, 12, 69, 88, 39, 47, 83, 41, 68, 39, 41, 41, 35, 91, 31, 77, 1, 90, 14, 88, 29, 75],
[32, 35, 4, 19, 58, 89, 100, 93, 7, 50, 42, 58, 50, 74, 41, 48, 20, 25, 97, 70, 58, 97, 25, 14, 31, 14, 92, 2, 46, 73, 31, 40, 64, 5, 51, 73, 82, 48, 54, 3, 12, 14, 64, 80, 63, 4, 85, 68, 48, 95, 15, 21, 42, 46, 68, 27, 26, 33, 53, 35, 98, 72, 53, 76, 17, 40, 84, 93, 28, 47, 4, 92, 40, 1, 74, 45, 63, 20, 36, 4, 77, 21, 6, 69, 81, 15, 40, 7, 56, 22, 61, 36, 19],
[91, 73, 9, 81, 81, 8, 37, 7, 40, 24, 65, 26, 84, 66, 65, 30, 30, 53, 27, 85, 78, 49, 43, 7, 65, 20, 27, 35, 53, 9, 10, 62, 21, 48, 8, 61, 40, 6, 35, 79, 8, 80, 21, 28, 32, 83, 32, 77, 71, 9, 56, 78, 46, 93, 66, 83, 16, 98, 41, 76, 78, 67, 46, 23, 45, 11, 80, 29, 21, 10, 10, 87, 49, 83, 83, 7, 81, 24, 21, 30, 59, 3, 34, 5, 25, 30, 76, 68, 2, 86, 97, 78, 45],
[96, 34, 46, 32, 17, 58, 41, 67, 92, 100, 33, 46, 79, 31, 45, 97, 43, 96, 67, 70, 9, 53, 73, 95, 64, 78, 82, 92, 57, 87, 42, 72, 88, 72, 68, 46, 89, 94, 75, 48, 10, 36, 65, 24, 74, 34, 16, 24, 52, 94, 74, 59, 30, 7, 38, 57, 8, 64, 9, 63, 43, 26, 25, 39, 58, 66, 38, 36, 70, 47, 12, 85, 94, 67, 52, 76, 23, 82, 98, 4, 1, 35, 43, 67, 94, 19, 31, 11, 4, 75, 54, 69, 45],
[40, 84, 74, 14, 24, 41, 82, 85, 58, 47, 48, 60, 6, 42, 66, 71, 19, 97, 46, 21, 24, 37, 23, 83, 3, 63, 45, 19, 17, 14, 54, 1, 95, 9, 98, 53, 80, 34, 57, 41, 74, 86, 99, 65, 31, 57, 45, 54, 9, 48, 76, 45, 67, 63, 23, 66, 21, 16, 29, 20, 87, 64, 9, 30, 96, 72, 33, 74, 34, 35, 94, 85, 28, 32, 100, 51, 14, 73, 30, 91, 46, 3, 29, 23, 73, 90, 20, 82, 25, 92, 37, 97, 61],
[12, 62, 75, 45, 27, 2, 41, 18, 31, 7, 95, 45, 17, 68, 95, 50, 95, 25, 95, 20, 46, 83, 27, 68, 17, 50, 81, 47, 56, 98, 18, 83, 27, 15, 18, 14, 30, 33, 62, 77, 74, 83, 37, 12, 71, 48, 59, 18, 61, 75, 83, 52, 44, 17, 87, 39, 54, 10, 27, 2, 99, 9, 5, 100, 41, 82, 52, 41, 30, 94, 30, 4, 50, 62, 78, 17, 4, 69, 16, 86, 61, 55, 6, 29, 45, 68, 47, 33, 84, 54, 55, 50, 8],
[36, 65, 84, 77, 86, 35, 21, 70, 10, 98, 1, 99, 30, 50, 43, 13, 13, 77, 22, 24, 18, 84, 63, 29, 68, 42, 96, 10, 44, 66, 51, 52, 74, 2, 56, 46, 97, 67, 35, 53, 96, 42, 63, 61, 88, 8, 30, 82, 15, 22, 27, 39, 61, 27, 41, 87, 77, 40, 21, 45, 18, 54, 95, 99, 64, 3, 100, 77, 32, 53, 8, 74, 87, 50, 23, 10, 17, 63, 12, 53, 46, 25, 80, 77, 93, 74, 27, 62, 42, 55, 88, 17, 42],
[63, 78, 48, 47, 26, 78, 65, 100, 23, 40, 66, 94, 10, 25, 96, 70, 26, 11, 11, 32, 34, 9, 7, 28, 12, 99, 7, 64, 70, 49, 78, 91, 47, 95, 56, 42, 9, 80, 39, 58, 77, 72, 56, 92, 55, 57, 38, 30, 52, 76, 79, 68, 61, 60, 2, 77, 44, 1, 75, 5, 36, 2, 98, 3, 68, 96, 47, 45, 55, 5, 59, 74, 13, 37, 84, 6, 73, 31, 61, 34, 54, 85, 28, 4, 6, 76, 74, 76, 58, 35, 49, 88, 56],
[89, 9, 40, 88, 10, 19, 48, 97, 100, 73, 57, 53, 5, 30, 78, 75, 85, 83, 36, 57, 52, 83, 16, 44, 15, 58, 42, 9, 97, 80, 65, 42, 17, 12, 9, 99, 90, 73, 62, 36, 63, 98, 9, 86, 86, 73, 29, 95, 99, 41, 44, 32, 49, 96, 75, 27, 60, 72, 66, 50, 51, 68, 98, 70, 27, 53, 61, 47, 26, 29, 48, 24, 35, 57, 56, 69, 36, 56, 69, 16, 22, 67, 59, 78, 2, 76, 66, 51, 90, 21, 30, 45, 51],
[36, 78, 70, 11, 38, 54, 46, 35, 76, 66, 98, 68, 42, 49, 28, 99, 73, 28, 44, 95, 35, 54, 88, 93, 29, 1, 57, 7, 63, 29, 23, 16, 8, 63, 72, 86, 99, 34, 71, 9, 37, 92, 26, 44, 33, 89, 52, 35, 68, 26, 86, 3, 36, 82, 14, 67, 43, 80, 26, 91, 41, 87, 74, 58, 100, 61, 18, 24, 87, 22, 29, 89, 12, 78, 51, 58, 72, 25, 19, 66, 47, 32, 28, 32, 80, 43, 21, 75, 16, 34, 89, 99, 88],
[28, 68, 67, 94, 2, 92, 15, 48, 4, 5, 26, 63, 100, 64, 29, 21, 97, 71, 35, 91, 88, 83, 91, 20, 4, 63, 53, 21, 17, 20, 57, 75, 60, 14, 31, 44, 54, 81, 11, 83, 19, 24, 6, 40, 5, 92, 63, 54, 78, 31, 71, 68, 98, 51, 61, 52, 67, 41, 69, 84, 5, 35, 9, 79, 47, 49, 95, 2, 42, 43, 41, 68, 75, 95, 14, 92, 100, 74, 44, 38, 100, 39, 71, 5, 40, 87, 3, 26, 3, 48, 89, 42, 74],
[46, 25, 97, 62, 20, 17, 35, 73, 23, 17, 53, 77, 68, 64, 71, 73, 5, 67, 23, 22, 79, 91, 91, 80, 42, 2, 47, 10, 34, 65, 92, 97, 68, 1, 55, 87, 83, 10, 49, 90, 100, 77, 43, 62, 79, 57, 38, 8, 52, 56, 46, 66, 89, 52, 59, 86, 79, 61, 13, 59, 41, 79, 48, 78, 79, 16, 88, 63, 53, 24, 89, 84, 93, 14, 39, 52, 31, 62, 13, 43, 78, 26, 67, 2, 27, 42, 14, 7, 92, 81, 48, 68, 86],
[25, 73, 65, 9, 21, 91, 52, 26, 51, 3, 57, 37, 69, 39, 44, 9, 35, 16, 81, 32, 85, 93, 57, 93, 75, 75, 94, 23, 41, 35, 1, 54, 90, 7, 6, 3, 2, 19, 7, 86, 10, 46, 36, 19, 76, 93, 53, 74, 64, 56, 43, 65, 4, 37, 20, 85, 92, 2, 88, 100, 15, 3, 81, 24, 78, 52, 72, 28, 33, 44, 24, 85, 78, 7, 47, 61, 44, 60, 43, 100, 17, 64, 79, 1, 25, 61, 91, 75, 80, 7, 75, 69, 27],
[28, 17, 89, 46, 55, 53, 49, 54, 43, 46, 97, 25, 65, 42, 37, 42, 68, 21, 92, 27, 22, 58, 5, 37, 16, 30, 12, 46, 38, 35, 31, 9, 24, 79, 48, 90, 22, 24, 85, 60, 32, 68, 52, 87, 88, 2, 67, 35, 83, 27, 86, 90, 5, 26, 53, 18, 62, 4, 8, 30, 9, 90, 24, 18, 20, 45, 93, 32, 88, 97, 27, 49, 8, 75, 37, 70, 64, 16, 58, 82, 45, 58, 10, 34, 80, 28, 58, 9, 29, 21, 29, 22, 94],
[7, 8, 97, 86, 73, 54, 48, 16, 51, 55, 78, 38, 39, 52, 59, 49, 13, 65, 48, 51, 7, 69, 69, 13, 100, 12, 100, 65, 68, 76, 68, 20, 80, 57, 82, 39, 2, 45, 52, 16, 33, 44, 47, 67, 99, 19, 90, 91, 16, 88, 55, 96, 9, 60, 74, 35, 80, 26, 41, 45, 32, 44, 41, 66, 85, 22, 36, 30, 4, 35, 51, 14, 60, 26, 9, 61, 88, 66, 89, 6, 36, 47, 67, 74, 80, 12, 30, 77, 18, 77, 34, 84, 4],
[61, 94, 50, 94, 46, 20, 70, 94, 53, 34, 13, 24, 99, 97, 64, 26, 26, 35, 78, 97, 23, 15, 1, 16, 46, 60, 63, 62, 71, 7, 16, 36, 19, 54, 44, 22, 100, 41, 29, 33, 47, 78, 22, 16, 17, 5, 71, 26, 25, 7, 15, 64, 74, 8, 54, 91, 21, 44, 45, 64, 85, 57, 8, 12, 31, 31, 59, 27, 17, 58, 100, 39, 75, 65, 31, 2, 69, 39, 75, 40, 38, 77, 80, 37, 19, 89, 41, 40, 26, 15, 90, 10, 27],
[56, 98, 20, 36, 85, 51, 3, 50, 61, 92, 56, 76, 99, 17, 50, 57, 91, 49, 17, 81, 51, 80, 41, 46, 83, 74, 20, 79, 81, 65, 36, 83, 25, 90, 81, 10, 32, 98, 55, 53, 27, 3, 99, 79, 42, 35, 80, 10, 8, 33, 15, 27, 89, 43, 42, 2, 49, 54, 1, 18, 48, 18, 62, 20, 77, 100, 89, 90, 30, 70, 22, 64, 70, 88, 79, 99, 7, 16, 2, 52, 48, 81, 37, 44, 39, 16, 79, 96, 81, 86, 96, 23, 70],
[79, 72, 14, 44, 80, 11, 75, 98, 42, 81, 62, 44, 82, 80, 37, 69, 38, 94, 26, 75, 78, 33, 17, 34, 20, 2, 5, 37, 24, 20, 1, 18, 87, 91, 64, 79, 9, 40, 68, 73, 67, 38, 64, 89, 56, 9, 67, 84, 26, 10, 49, 74, 77, 11, 90, 2, 80, 91, 9, 12, 28, 83, 83, 23, 2, 9, 96, 9, 41, 59, 79, 91, 16, 66, 32, 18, 23, 12, 1, 85, 66, 44, 88, 81, 17, 89, 11, 75, 63, 73, 1, 18, 60],
[41, 20, 40, 20, 14, 60, 13, 28, 43, 94, 41, 21, 12, 95, 94, 68, 93, 88, 64, 6, 54, 64, 18, 34, 2, 1, 27, 36, 52, 25, 42, 8, 8, 87, 38, 67, 57, 83, 72, 68, 51, 34, 73, 49, 92, 96, 50, 89, 50, 34, 62, 87, 34, 37, 10, 31, 18, 35, 15, 54, 69, 79, 45, 52, 23, 100, 85, 44, 32, 84, 65, 37, 94, 13, 29, 38, 95, 47, 81, 68, 49, 98, 12, 48, 53, 2, 69, 1, 28, 54, 64, 69, 3],
[79, 66, 23, 22, 97, 43, 19, 81, 45, 95, 32, 63, 93, 29, 45, 62, 25, 73, 92, 22, 17, 5, 44, 46, 19, 69, 56, 42, 88, 85, 10, 96, 48, 56, 41, 5, 61, 12, 28, 1, 81, 18, 96, 66, 38, 59, 84, 16, 62, 29, 82, 55, 65, 92, 77, 79, 6, 65, 41, 52, 20, 83, 2, 11, 62, 50, 20, 16, 9, 40, 22, 45, 64, 70, 97, 77, 53, 46, 99, 31, 44, 22, 34, 99, 88, 17, 45, 22, 7, 48, 47, 7, 79],
[94, 80, 48, 79, 98, 45, 46, 32, 34, 55, 8, 6, 94, 88, 9, 25, 66, 52, 82, 10, 7, 56, 88, 91, 96, 6, 48, 29, 49, 62, 3, 86, 31, 67, 45, 35, 12, 90, 6, 46, 80, 80, 97, 15, 72, 52, 15, 41, 43, 46, 99, 23, 10, 32, 15, 40, 16, 59, 56, 45, 5, 49, 80, 37, 95, 56, 98, 48, 41, 2, 53, 34, 65, 84, 63, 4, 80, 85, 80, 45, 36, 11, 77, 66, 42, 13, 57, 75, 73, 70, 67, 17, 39]
]
find(toll_map, (20, 35), (70, 80))

['up',
 'right',
 'right',
 'right',
 'down',
 'down',
 'right',
 'right',
 'down',
 'right',
 'down',
 'right',
 'down',
 'down',
 'down',
 'right',
 'down',
 'down',
 'down',
 'down',
 'down',
 'right',
 'right',
 'right',
 'down',
 'down',
 'right',
 'down',
 'down',
 'down',
 'down',
 'down',
 'right',
 'down',
 'down',
 'down',
 'right',
 'down',
 'right',
 'down',
 'right',
 'right',
 'down',
 'right',
 'right',
 'right',
 'right',
 'right',
 'right',
 'down',
 'down',
 'down',
 'down',
 'down',
 'right',
 'right',
 'right',
 'down',
 'down',
 'right',
 'down',
 'down',
 'down',
 'down',
 'right',
 'down',
 'right',
 'down',
 'down',
 'down',
 'right',
 'right',
 'down',
 'right',
 'right',
 'right',
 'down',
 'right',
 'right',
 'right',
 'down',
 'down',
 'right',
 'right',
 'right',
 'down',
 'down',
 'right',
 'down',
 'down',
 'down',
 'right',
 'right',
 'right',
 'down',
 'down',
 'right']

In [ ]:
findhq(toll_map, (20, 35), (70, 80))